[![AWS SDK for pandas](_static/logo.png "AWS SDK for pandas")](https://github.com/aws/aws-sdk-pandas)

# 35 - OpenSearch Serverless

Amazon OpenSearch Serverless is an on-demand serverless configuration for Amazon OpenSearch Service.

### Create collection

A collection in Amazon OpenSearch Serverless is a logical grouping of one or more indexes that represent an analytics workload.

Collections must have an assigned encryption policy, network policy, and a matching data access policy that grants permission to its resources.

### Define policies

In [6]:
import awswrangler as wr

data_access_policy = [
    {
        "Rules": [
            {
                "ResourceType": "index",
                "Resource": [
                    "index/*/*",
                ],
                "Permission": [
                    "aoss:*",
                ],
            },
            {
                "ResourceType": "collection",
                "Resource": [
                    "collection/*",
                ],
                "Permission": [
                    "aoss:*",
                ],
            },
        ],
        "Principal": [
            wr.sts.get_current_identity_arn(),
        ],
    }
]

AWS SDK for pandas can create default network and encryption policies based on the user input. By default, the network policy allows public access to the collection, and the encryption policy encrypts the collection using AWS-managed KMS key.

Now, create the collection, and the corresponding network, encryption, and data access policy:

In [9]:
collection = wr.opensearch.create_collection(
    name="my-collection",
    data_policy=data_access_policy,
)

collection_endpoint = collection["collectionEndpoint"]

Now you can connect to the collection endpoint:

In [10]:
client = wr.opensearch.connect(host=collection_endpoint)

## Create index

To create an index, run:

In [16]:
index="my-index-1"

wr.opensearch.create_index(
    client=client,
    index=index,
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-index-1'}

## Index and search documents

To index and search documents:

In [17]:
wr.opensearch.index_documents(
    client,
    documents=[{"_id": "1", "name": "John"}, {"_id": "2", "name": "George"}, {"_id": "3", "name": "Julia"}],
    index=index,
)

Indexing: 100% (3/3)|####################################|Elapsed Time: 0:00:01

{'success': 3, 'errors': []}

In [19]:
wr.opensearch.search(
    client,
    index=index,
)

,_id,name
0,3,Julia
1,2,George
2,1,John


For more examples, refer to the OpenSearch tutorial.